In [1]:
using Distributed

In [2]:
addprocs(1) # リモートのワーカプロセスを追加
# このほかにjuliaを起動するときに `$ julia -p 2` とすることでマスタプロセスと2つのワーカプロセスが起動する。

1-element Array{Int64,1}:
 2

In [3]:
Distributed.myid() # マスタワーカプロセスのId番号

1

In [4]:
workers() # リモートワーカプロセスのID

1-element Array{Int64,1}:
 2

In [6]:
res = @spawnat 2 myid()

Future(2, 1, 3, nothing)

In [7]:
fetch(res)

2

In [8]:
# リモートプロセス上でジョブを実行
# スリープしたあとランダムな値を返すだけ
remote_f = function(s::Int=3)
    println("Worker $(myid()) will sleep for $s seconds")
    sleep(s)
    val = rand(1:1000)
    println("Completed worker $(myid()) - return $val")
    return val
end

#5 (generic function with 2 methods)

In [9]:
@fetchfrom 2 remote_f(4) # ID2のワーカで実行

      From worker 2:	Worker 2 will sleep for 4 seconds
      From worker 2:	Completed worker 2 - return 531


531

In [23]:
function run_timeout(timeout::Int, f::Function, params...)    
    wid = addprocs(1)[1]
    result = RemoteChannel(()->Channel{Tuple}(1));
    @spawnat wid put!(result, (f(params...), myid()))
    res = nothing
    time_elapsed = 0.0
    while time_elapsed < timeout && !isready(result)
        sleep(0.25)
        time_elapsed += 0.25
    end
    if !isready(result)
        println("Not completed! Computation at $wid will be terminated!")
    else
        res = take!(result)
    end
    rmprocs(wid);
    return res
end

run_timeout (generic function with 1 method)

In [24]:
run_timeout(3, remote_f, 2)

      From worker 3:	Worker 3 will sleep for 2 seconds
      From worker 3:	Completed worker 3 - return 595


(595, 3)

In [29]:
run_timeout(3, remote_f, 10)

      From worker 6:	Worker 6 will sleep for 10 seconds
Not completed! Computation at 6 will be terminated!


In [27]:
workers()

1-element Array{Int64,1}:
 2

In [30]:
# リモートプロセス上で関数を実行する場合は各リモートプロセス上で定義されていなければならない
# または, @everywhere マクロを用いればよい

In [ ]:
## multithread

In [1]:
Threads.nthreads()

1

In [3]:
using DataFrames, BenchmarkTools, Random, Statistics

In [4]:
Random.seed!(0)
N=100_000
const data = DataFrame(rowtype=rand(1:12,N))
data.x1 = data.rowtype .* randn(N)

100000-element Array{Float64,1}:
   9.421511016490118
  -4.222008285708796
   1.1451481790872424
   0.6222675834409342
   8.848740553387213
   3.4493258003813305
  -8.282638203983423
   0.3470873789692497
   3.7722799618130427
   4.5430278996270905
   1.329011567665244
  20.252304526349114
  -6.042228472162345
   ⋮
  -1.6635711096492036
  19.117817531821682
  -1.6948429867987025
 -20.52158031179198
  -4.4710010700941885
  -8.262883044729193
  -1.2888780603901364
  -2.429057365920029
  -0.6817034496730191
   2.1599875669117377
 -10.124861071547029
  -6.339531210064506

In [6]:
first(data, 5)

,rowtype,x1
,Int64,Float64
1,9,9.42151
2,2,-4.22201
3,6,1.14515
4,2,0.622268
5,12,8.84874
